In [1]:
# import _init_paths
import argparse
import os
import time

import random
import numpy as np
import yaml
import copy
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from datasets.linemod.dataset import PoseDataset as PoseDataset_linemod
from lib.network import PoseNet, PoseRefineNet
from lib.loss import Loss
from lib.loss_refiner import Loss_refine
from lib.transformations import euler_matrix, quaternion_matrix, quaternion_from_matrix
from lib.knn.__init__ import KNearestNeighbor

/home/amoghtiwari/DenseFusion/lib/transformations.py:1912: UserWarning: failed to import module _transformations
  warnings.warn('failed to import module %s' % name)


In [2]:
opt_dataset_root = "/ssd_scratch/cvit/amoghtiwari/densefusion/data/Linemod_preprocessed"
opt_model = "/ssd_scratch/cvit/amoghtiwari/densefusion/checkpoints/trained_checkpoints/linemod/pose_model_9_0.01310166542980859.pth"
opt_refine_model = "/ssd_scratch/cvit/amoghtiwari/densefusion/checkpoints/trained_checkpoints/linemod/pose_refine_model_493_0.006761023565178073.pth"


In [3]:
num_objects = 13
objlist = [1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15]
num_points = 500
iteration = 4
bs = 1
dataset_config_dir = 'datasets/linemod/dataset_config'
output_result_dir = 'experiments/eval_result/linemod'
knn = KNearestNeighbor(1)


In [4]:
estimator = PoseNet(num_points = num_points, num_obj = num_objects)
estimator.cuda()
refiner = PoseRefineNet(num_points = num_points, num_obj = num_objects)
refiner.cuda()
estimator.load_state_dict(torch.load(opt_model))
refiner.load_state_dict(torch.load(opt_refine_model))
estimator.eval()
refiner.eval()


PoseRefineNet(
  (feat): PoseRefineNetFeat(
    (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
    (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (e_conv1): Conv1d(32, 64, kernel_size=(1,), stride=(1,))
    (e_conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (conv5): Conv1d(384, 512, kernel_size=(1,), stride=(1,))
    (conv6): Conv1d(512, 1024, kernel_size=(1,), stride=(1,))
    (ap1): AvgPool1d(kernel_size=(500,), stride=(500,), padding=(0,))
  )
  (conv1_r): Linear(in_features=1024, out_features=512, bias=True)
  (conv1_t): Linear(in_features=1024, out_features=512, bias=True)
  (conv2_r): Linear(in_features=512, out_features=128, bias=True)
  (conv2_t): Linear(in_features=512, out_features=128, bias=True)
  (conv3_r): Linear(in_features=128, out_features=52, bias=True)
  (conv3_t): Linear(in_features=128, out_features=39, bias=True)
)

In [5]:
testdataset = PoseDataset_linemod('eval', num_points, False, opt_dataset_root, 0.0, True)
testdataloader = torch.utils.data.DataLoader(testdataset, batch_size=1, shuffle=False, num_workers=10)

/home/amoghtiwari/DenseFusion/datasets/linemod/dataset.py:66: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.meta[item] = yaml.load(meta_file)


Object 1 buffer loaded
Object 2 buffer loaded
Object 4 buffer loaded
Object 5 buffer loaded
Object 6 buffer loaded
Object 8 buffer loaded
Object 9 buffer loaded
Object 10 buffer loaded
Object 11 buffer loaded
Object 12 buffer loaded
Object 13 buffer loaded
Object 14 buffer loaded
Object 15 buffer loaded


In [6]:
sym_list = testdataset.get_sym_list()
num_points_mesh = testdataset.get_num_points_mesh()

In [7]:
print(sym_list)

[7, 8]


In [8]:
criterion = Loss(num_points_mesh, sym_list)
criterion_refine = Loss_refine(num_points_mesh, sym_list)

/home/amoghtiwari/miniconda3/envs/densefusion/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='elementwise_mean' instead.
  warnings.warn(warning.format(ret))


In [9]:
diameter = []
meta_file = open('{0}/models_info.yml'.format(dataset_config_dir), 'r')
meta = yaml.load(meta_file)
for obj in objlist:
    diameter.append(meta[obj]['diameter'] / 1000.0 * 0.1)
print(diameter)

[0.010209865663, 0.024750624233, 0.017249224865, 0.020140358597000002, 0.015454551808, 0.026147178102, 0.010899920102000001, 0.016462758847999998, 0.017588933422000002, 0.014554287471000002, 0.027807811733, 0.028260129399, 0.021235825148]


/home/amoghtiwari/miniconda3/envs/densefusion/lib/python3.6/site-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


## With Refinement

In [10]:
fw = open('{0}/eval_result_logs_with_refinement.txt'.format(output_result_dir), 'w')

In [11]:
success_count = [0 for i in range(num_objects)]
num_count = [0 for i in range(num_objects)]
fw = open('{0}/eval_result_logs_with_refinement.txt'.format(output_result_dir), 'w')

less_than_two = np.zeros((15,))
total = np.zeros((15,))

start = time.time()
for i, data in enumerate(testdataloader, 0):
#     print(i, data)
    points, choose, img, target, model_points, idx = data
    if len(points.size()) == 2:
        print('No.{0} NOT Pass! Lost detection!'.format(i))
        fw.write('No.{0} NOT Pass! Lost detection!\n'.format(i))
        continue
    points, choose, img, target, model_points, idx = Variable(points).cuda(), \
                                                     Variable(choose).cuda(), \
                                                     Variable(img).cuda(), \
                                                     Variable(target).cuda(), \
                                                     Variable(model_points).cuda(), \
                                                     Variable(idx).cuda()

    pred_r, pred_t, pred_c, emb = estimator(img, points, choose, idx)
    pred_r = pred_r / torch.norm(pred_r, dim=2).view(1, num_points, 1)
    pred_c = pred_c.view(bs, num_points)
    how_max, which_max = torch.max(pred_c, 1)
    pred_t = pred_t.view(bs * num_points, 1, 3)

    my_r = pred_r[0][which_max[0]].view(-1).cpu().data.numpy()
    my_t = (points.view(bs * num_points, 1, 3) + pred_t)[which_max[0]].view(-1).cpu().data.numpy()
    my_pred = np.append(my_r, my_t)

    """ Below code is for refinement """

    for ite in range(0, iteration):
        T = Variable(torch.from_numpy(my_t.astype(np.float32))).cuda().view(1, 3).repeat(num_points, 1).contiguous().view(1, num_points, 3)
        my_mat = quaternion_matrix(my_r)
        R = Variable(torch.from_numpy(my_mat[:3, :3].astype(np.float32))).cuda().view(1, 3, 3)
        my_mat[0:3, 3] = my_t
        
        new_points = torch.bmm((points - T), R).contiguous()
        pred_r, pred_t = refiner(new_points, emb, idx)
        pred_r = pred_r.view(1, 1, -1)
        pred_r = pred_r / (torch.norm(pred_r, dim=2).view(1, 1, 1))
        my_r_2 = pred_r.view(-1).cpu().data.numpy()
        my_t_2 = pred_t.view(-1).cpu().data.numpy()
        my_mat_2 = quaternion_matrix(my_r_2)
        my_mat_2[0:3, 3] = my_t_2

        my_mat_final = np.dot(my_mat, my_mat_2)
        my_r_final = copy.deepcopy(my_mat_final)
        my_r_final[0:3, 3] = 0
        my_r_final = quaternion_from_matrix(my_r_final, True)
        my_t_final = np.array([my_mat_final[0][3], my_mat_final[1][3], my_mat_final[2][3]])

        my_pred = np.append(my_r_final, my_t_final)
        my_r = my_r_final
        my_t = my_t_final

    # Here 'my_pred' is the final pose estimation result after refinement ('my_r': quaternion, 'my_t': translation)

    model_points = model_points[0].cpu().detach().numpy()
    my_r = quaternion_matrix(my_r)[:3, :3]
    pred = np.dot(model_points, my_r.T) + my_t
    target = target[0].cpu().detach().numpy()

    if idx[0].item() in sym_list:
        pred = torch.from_numpy(pred.astype(np.float32)).cuda().transpose(1, 0).contiguous()
        target = torch.from_numpy(target.astype(np.float32)).cuda().transpose(1, 0).contiguous()
        inds = knn(target.unsqueeze(0), pred.unsqueeze(0))
        target = torch.index_select(target, 1, inds.view(-1) - 1)
        dis = torch.mean(torch.norm((pred.transpose(1, 0) - target.transpose(1, 0)), dim=1), dim=0).item()
    else:
        dis = np.mean(np.linalg.norm(pred - target, axis=1))

    if dis/100 < diameter[idx[0].item()]:
        success_count[idx[0].item()] += 1
        print('No.{0} Pass! Distance: {1}'.format(i, dis))
        fw.write('No.{0} Pass! Distance: {1}\n'.format(i, dis))
    else:
        print('No.{0} NOT Pass! Distance: {1}'.format(i, dis))
        fw.write('No.{0} NOT Pass! Distance: {1}\n'.format(i, dis))
    if dis < 0.02:
        less_than_two[idx]+= 1
    total[idx]+= 1
    
    num_count[idx[0].item()] += 1

end = time.time()

/home/amoghtiwari/miniconda3/envs/densefusion/lib/python3.6/site-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/home/amoghtiwari/miniconda3/envs/densefusion/lib/python3.6/site-packages/torch/nn/functional.py:1961: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/amoghtiwari/miniconda3/envs/densefusion/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")
/home/amoghtiwari/miniconda3/envs/dense

No.0 Pass! Distance: 0.003915328139615649
No.1 Pass! Distance: 0.0032644904169231914
No.2 Pass! Distance: 0.0061389586914043105
No.3 Pass! Distance: 0.005557423069544938
No.4 Pass! Distance: 0.0028832851952111296
No.5 Pass! Distance: 0.0036587351710748563
No.6 Pass! Distance: 0.004557734526110883
No.7 Pass! Distance: 0.0032907602537871686
No.8 Pass! Distance: 0.0014744398386504747
No.9 Pass! Distance: 0.00589097977685614
No.10 Pass! Distance: 0.004884544556543459
No.11 Pass! Distance: 0.0037897333246572486
No.12 Pass! Distance: 0.006360595446305204
No.13 Pass! Distance: 0.08328723617071902
No.14 Pass! Distance: 0.006496485591783965
No.15 Pass! Distance: 0.0018456452141544526
No.16 Pass! Distance: 0.004198723208722518
No.17 Pass! Distance: 0.0027924186368420622
No.18 Pass! Distance: 0.0037365693117891014
No.19 Pass! Distance: 0.004601712553517994
No.20 Pass! Distance: 0.004205996808884352
No.21 Pass! Distance: 0.004750270437426069
No.22 Pass! Distance: 0.006449175897239247
No.23 Pass! D

No.194 Pass! Distance: 0.003547523098821674
No.195 Pass! Distance: 0.002764250036432302
No.196 Pass! Distance: 0.0017549995248592868
No.197 Pass! Distance: 0.0022949366331350396
No.198 Pass! Distance: 0.0028925153696117965
No.199 Pass! Distance: 0.007951715980835026
No.200 Pass! Distance: 0.018501289162506644
No.201 Pass! Distance: 0.004927336107017658
No.202 Pass! Distance: 0.0058904488644410035
No.203 Pass! Distance: 0.007592675782989005
No.204 Pass! Distance: 0.003136784370438893
No.205 Pass! Distance: 0.009353126624628214
No.206 Pass! Distance: 0.016548282976336842
No.207 Pass! Distance: 0.00259979834341256
No.208 Pass! Distance: 0.0023284362030325813
No.209 Pass! Distance: 0.003806519160835392
No.210 Pass! Distance: 0.0023156449746660063
No.211 Pass! Distance: 0.0027854656789701496
No.212 Pass! Distance: 0.0018540497517349904
No.213 Pass! Distance: 0.005011531817387974
No.214 Pass! Distance: 0.004688553161972547
No.215 Pass! Distance: 0.0030085890875882902
No.216 Pass! Distance: 0

No.388 Pass! Distance: 0.0032891024049154336
No.389 Pass! Distance: 0.0023160272227589858
No.390 Pass! Distance: 0.019238229207270217
No.391 Pass! Distance: 0.0137525469392024
No.392 Pass! Distance: 0.032700192523051334
No.393 Pass! Distance: 0.0026725764891015994
No.394 Pass! Distance: 0.002014512106151944
No.395 Pass! Distance: 0.003066686594038456
No.396 Pass! Distance: 0.0019685374688674317
No.397 Pass! Distance: 0.002451763354342815
No.398 Pass! Distance: 0.002289821790279273
No.399 Pass! Distance: 0.0017560709805981777
No.400 Pass! Distance: 0.0025186180408699748
No.401 Pass! Distance: 0.0019198065045563032
No.402 Pass! Distance: 0.002462156717661282
No.403 Pass! Distance: 0.00141852181530254
No.404 Pass! Distance: 0.002049278703646242
No.405 Pass! Distance: 0.003012017952320649
No.406 Pass! Distance: 0.0020666053739481554
No.407 Pass! Distance: 0.0031996457228744505
No.408 Pass! Distance: 0.004160040129677814
No.409 Pass! Distance: 0.0025533221734729957
No.410 Pass! Distance: 0.

No.574 Pass! Distance: 0.004564243237571221
No.575 Pass! Distance: 0.005655351283659582
No.576 Pass! Distance: 0.002197276369649084
No.577 Pass! Distance: 0.005603584547873189
No.578 Pass! Distance: 0.0031633091255862116
No.579 Pass! Distance: 0.0023172853694218036
No.580 Pass! Distance: 0.003433527124129412
No.581 Pass! Distance: 0.004485884403994176
No.582 Pass! Distance: 0.005255657417974374
No.583 Pass! Distance: 0.0012971668435318713
No.584 NOT Pass! Lost detection!
No.585 Pass! Distance: 0.003725456150764089
No.586 Pass! Distance: 0.005813644405221098
No.587 Pass! Distance: 0.005013499914831071
No.588 Pass! Distance: 0.0030052844054158125
No.589 Pass! Distance: 0.0020738567221841252
No.590 Pass! Distance: 0.007816155469035009
No.591 Pass! Distance: 0.008442524249499303
No.592 Pass! Distance: 0.005486793605183869
No.593 Pass! Distance: 0.002436206339439856
No.594 Pass! Distance: 0.0023578826752072048
No.595 Pass! Distance: 0.0029352484420070418
No.596 Pass! Distance: 0.00285078114

...

No.12577 Pass! Distance: 0.004302825891668531
No.12578 Pass! Distance: 0.012874923325212567
No.12579 Pass! Distance: 0.006588163902361237
No.12580 Pass! Distance: 0.004650289891114893
No.12581 Pass! Distance: 0.0034029926342569504
No.12582 Pass! Distance: 0.0025488911359607042
No.12583 Pass! Distance: 0.0034706492149416627
No.12584 Pass! Distance: 0.003879413726192807
No.12585 Pass! Distance: 0.005783323888130478
No.12586 Pass! Distance: 0.0026711222839430576
No.12587 Pass! Distance: 0.006849927631995719
No.12588 Pass! Distance: 0.0033921752683206243
No.12589 Pass! Distance: 0.009124397196148468
No.12590 Pass! Distance: 0.003011643946094513
No.12591 Pass! Distance: 0.004915528218632369
No.12592 Pass! Distance: 0.0035677435729623112
No.12593 Pass! Distance: 0.008120292964572047
No.12594 Pass! Distance: 0.005436854469536653
No.12595 Pass! Distance: 0.008947408323335819
No.12596 Pass! Distance: 0.004822384978249841
No.12597 Pass! Distance: 0.0029543443804771296
No.12598 Pass! Distance: 0.

No.12762 Pass! Distance: 0.006608531467548318
No.12763 Pass! Distance: 0.002547811470997747
No.12764 Pass! Distance: 0.002802498078566319
No.12765 Pass! Distance: 0.006586103675946126
No.12766 Pass! Distance: 0.004228954567211528
No.12767 Pass! Distance: 0.060516128768426804
No.12768 Pass! Distance: 0.002376827301476525
No.12769 Pass! Distance: 0.005108445151552105
No.12770 Pass! Distance: 0.005770153362531584
No.12771 Pass! Distance: 0.003471204915781256
No.12772 Pass! Distance: 0.007848783828389036
No.12773 Pass! Distance: 0.05389931911695189
No.12774 Pass! Distance: 0.002341663978916339
No.12775 Pass! Distance: 0.012371292529314312
No.12776 Pass! Distance: 0.0043122514691257925
No.12777 Pass! Distance: 0.004824776305537325
No.12778 Pass! Distance: 0.004172559491272572
No.12779 Pass! Distance: 0.009937116974887581
No.12780 Pass! Distance: 0.004834267429630907
No.12781 Pass! Distance: 0.003974942226248034
No.12782 Pass! Distance: 0.0051646758766763735
No.12783 Pass! Distance: 0.005666

No.12942 Pass! Distance: 0.017579015733986215
No.12943 Pass! Distance: 0.014844487424554013
No.12944 Pass! Distance: 0.00719837312180966
No.12945 Pass! Distance: 0.007214107110839027
No.12946 Pass! Distance: 0.0040011051622525
No.12947 Pass! Distance: 0.003089767572848956
No.12948 Pass! Distance: 0.0028602652229891526
No.12949 Pass! Distance: 0.00271255396125344
No.12950 Pass! Distance: 0.005131124850522831
No.12951 Pass! Distance: 0.005722811478325432
No.12952 Pass! Distance: 0.006827970170145426
No.12953 Pass! Distance: 0.0055656308601078775
No.12954 Pass! Distance: 0.0029240004513239617
No.12955 Pass! Distance: 0.06744380228934697
No.12956 Pass! Distance: 0.0018996951516817848
No.12957 Pass! Distance: 0.006284589379834206
No.12958 Pass! Distance: 0.004803867287374235
No.12959 Pass! Distance: 0.014830556719102462
No.12960 Pass! Distance: 0.005950013793370319
No.12961 Pass! Distance: 0.004089644411691149
No.12962 Pass! Distance: 0.0025177783319378423
No.12963 Pass! Distance: 0.0036543

No.13123 Pass! Distance: 0.009030238519802548
No.13124 Pass! Distance: 0.008031485193227945
No.13125 Pass! Distance: 0.001617459433560919
No.13126 Pass! Distance: 0.0048596463396095935
No.13127 Pass! Distance: 0.0036034690532203333
No.13128 Pass! Distance: 0.006002567664658228
No.13129 Pass! Distance: 0.006058716324351112
No.13130 Pass! Distance: 0.007867159604410132
No.13131 Pass! Distance: 0.0029265722174200023
No.13132 Pass! Distance: 0.0017547535440065461
No.13133 Pass! Distance: 0.0026327150765698103
No.13134 Pass! Distance: 0.0015079874516864748
No.13135 Pass! Distance: 0.002077227677986766
No.13136 Pass! Distance: 0.004174358827375942
No.13137 Pass! Distance: 0.010270629636633203
No.13138 Pass! Distance: 0.00858063766679421
No.13139 Pass! Distance: 0.004601118332597521
No.13140 Pass! Distance: 0.004668117073390492
No.13141 Pass! Distance: 0.004865735576226871
No.13142 Pass! Distance: 0.004420131891794578
No.13143 Pass! Distance: 0.002476336753439312
No.13144 Pass! Distance: 0.00

No.13301 Pass! Distance: 0.007266850270835016
No.13302 Pass! Distance: 0.004416379978557473
No.13303 Pass! Distance: 0.009291359062900139
No.13304 Pass! Distance: 0.006339140604075588
No.13305 Pass! Distance: 0.011996501192496017
No.13306 Pass! Distance: 0.0034564643518063703
No.13307 Pass! Distance: 0.00644633348134243
No.13308 Pass! Distance: 0.016132192876188077
No.13309 Pass! Distance: 0.0035955974341827794
No.13310 Pass! Distance: 0.0019107027221479144
No.13311 Pass! Distance: 0.0034037557254761926
No.13312 Pass! Distance: 0.008164945388154774
No.13313 Pass! Distance: 0.008485137346392146
No.13314 Pass! Distance: 0.011002598714446967
No.13315 Pass! Distance: 0.01365216230685478
No.13316 Pass! Distance: 0.009169986855060716
No.13317 Pass! Distance: 0.008079019800995174
No.13318 Pass! Distance: 0.010299778766912964
No.13319 Pass! Distance: 0.002280633645134884
No.13320 Pass! Distance: 0.004251217463789835
No.13321 Pass! Distance: 0.0029845683739319127
No.13322 Pass! Distance: 0.0023

### Evaluation

In [14]:
print("The time of execution of above program is :", end-start)
fw.write(f"The time of execution of above program is :{end-start}")

The time of execution of above program is : 338.87715220451355


61

In [15]:
obj_ids = [0,1,2,4,5,6,8,9,10,11,12,13,14]
for i in range(len(obj_ids)):
    print(f"obj_id: {obj_ids[i]}; Success Rate: {100*less_than_two[i] / total[i]}")
    fw.write(f"obj_id: {obj_ids[i]}; Success Rate: {100*less_than_two[i] / total[i]}")

obj_id: 0; Success Rate: 97.61677788369876
obj_id: 1; Success Rate: 92.91949563530552
obj_id: 2; Success Rate: 97.25490196078431
obj_id: 4; Success Rate: 94.19291338582677
obj_id: 5; Success Rate: 97.0059880239521
obj_id: 6; Success Rate: 86.32309217046581
obj_id: 8; Success Rate: 96.52582159624413
obj_id: 9; Success Rate: 100.0
obj_id: 10; Success Rate: 99.9034749034749
obj_id: 11; Success Rate: 94.95718363463368
obj_id: 12; Success Rate: 97.13993871297242
obj_id: 13; Success Rate: 96.16122840690979
obj_id: 14; Success Rate: 95.1969260326609


In [17]:
print("Overall success rate ", 100 * sum(less_than_two) / sum(total))
fw.write(f"Overall success rate {100 * sum(less_than_two) / sum(total)}")

Overall success rate  95.80722172485825


38

In [18]:
fw.close()
less_than_two_old = less_than_two
total_old = total

## Without Refinement

In [19]:
success_count = [0 for i in range(num_objects)]
num_count = [0 for i in range(num_objects)]
fw = open('{0}/eval_result_logs_without_refinement.txt'.format(output_result_dir), 'w')

less_than_two = np.zeros((15,))
total = np.zeros((15,))

start = time.time()
for i, data in enumerate(testdataloader, 0):
#     print(i, data)
    points, choose, img, target, model_points, idx = data
    if len(points.size()) == 2:
        print('No.{0} NOT Pass! Lost detection!'.format(i))
        fw.write('No.{0} NOT Pass! Lost detection!\n'.format(i))
        continue
    points, choose, img, target, model_points, idx = Variable(points).cuda(), \
                                                     Variable(choose).cuda(), \
                                                     Variable(img).cuda(), \
                                                     Variable(target).cuda(), \
                                                     Variable(model_points).cuda(), \
                                                     Variable(idx).cuda()

    pred_r, pred_t, pred_c, emb = estimator(img, points, choose, idx)
#     print(idx)
#     break
    pred_r = pred_r / torch.norm(pred_r, dim=2).view(1, num_points, 1)
    pred_c = pred_c.view(bs, num_points)
    how_max, which_max = torch.max(pred_c, 1)
    pred_t = pred_t.view(bs * num_points, 1, 3)

    my_r = pred_r[0][which_max[0]].view(-1).cpu().data.numpy()
    my_t = (points.view(bs * num_points, 1, 3) + pred_t)[which_max[0]].view(-1).cpu().data.numpy()
    my_pred = np.append(my_r, my_t)

    """    Below code is for refinement """
#     for ite in range(0, iteration):
#         T = Variable(torch.from_numpy(my_t.astype(np.float32))).cuda().view(1, 3).repeat(num_points, 1).contiguous().view(1, num_points, 3)
#         my_mat = quaternion_matrix(my_r)
#         R = Variable(torch.from_numpy(my_mat[:3, :3].astype(np.float32))).cuda().view(1, 3, 3)
#         my_mat[0:3, 3] = my_t
        
#         new_points = torch.bmm((points - T), R).contiguous()
#         pred_r, pred_t = refiner(new_points, emb, idx)
#         pred_r = pred_r.view(1, 1, -1)
#         pred_r = pred_r / (torch.norm(pred_r, dim=2).view(1, 1, 1))
#         my_r_2 = pred_r.view(-1).cpu().data.numpy()
#         my_t_2 = pred_t.view(-1).cpu().data.numpy()
#         my_mat_2 = quaternion_matrix(my_r_2)
#         my_mat_2[0:3, 3] = my_t_2

#         my_mat_final = np.dot(my_mat, my_mat_2)
#         my_r_final = copy.deepcopy(my_mat_final)
#         my_r_final[0:3, 3] = 0
#         my_r_final = quaternion_from_matrix(my_r_final, True)
#         my_t_final = np.array([my_mat_final[0][3], my_mat_final[1][3], my_mat_final[2][3]])

#         my_pred = np.append(my_r_final, my_t_final)
#         my_r = my_r_final
#         my_t = my_t_final

#     # Here 'my_pred' is the final pose estimation result after refinement ('my_r': quaternion, 'my_t': translation)

    model_points = model_points[0].cpu().detach().numpy()
    my_r = quaternion_matrix(my_r)[:3, :3]
    pred = np.dot(model_points, my_r.T) + my_t
    target = target[0].cpu().detach().numpy()

    if idx[0].item() in sym_list:
        pred = torch.from_numpy(pred.astype(np.float32)).cuda().transpose(1, 0).contiguous()
        target = torch.from_numpy(target.astype(np.float32)).cuda().transpose(1, 0).contiguous()
        inds = knn(target.unsqueeze(0), pred.unsqueeze(0))
        target = torch.index_select(target, 1, inds.view(-1) - 1)
        dis = torch.mean(torch.norm((pred.transpose(1, 0) - target.transpose(1, 0)), dim=1), dim=0).item()
    else:
        dis = np.mean(np.linalg.norm(pred - target, axis=1))

    if dis/100 < diameter[idx[0].item()]:
        success_count[idx[0].item()] += 1
        print('No.{0} Pass! Distance: {1}'.format(i, dis))
        fw.write('No.{0} Pass! Distance: {1}\n'.format(i, dis))
    else:
        print('No.{0} NOT Pass! Distance: {1}'.format(i, dis))
        fw.write('No.{0} NOT Pass! Distance: {1}\n'.format(i, dis))
    if dis < 0.02:
        less_than_two[idx]+= 1
    total[idx]+= 1
    
    num_count[idx[0].item()] += 1

end = time.time()

No.0 Pass! Distance: 0.005047377628032251
No.1 Pass! Distance: 0.006319127512505292
No.2 Pass! Distance: 0.0076935918193199405
No.3 Pass! Distance: 0.004998787704688944
No.4 Pass! Distance: 0.004323573822714671
No.5 Pass! Distance: 0.0039758147252363435
No.6 Pass! Distance: 0.003477903140943007
No.7 Pass! Distance: 0.0027329746096111645
No.8 Pass! Distance: 0.004965289355104457
No.9 Pass! Distance: 0.007925282197603998
No.10 Pass! Distance: 0.008852626564210386
No.11 Pass! Distance: 0.008137073071681291
No.12 Pass! Distance: 0.009695593822648984
No.13 Pass! Distance: 0.0649231466887941
No.14 Pass! Distance: 0.01084953909648505
No.15 Pass! Distance: 0.013557397805248764
No.16 Pass! Distance: 0.009645255021960479
No.17 Pass! Distance: 0.007245509669970911
No.18 Pass! Distance: 0.009067104826477247
No.19 Pass! Distance: 0.008765521170833932
No.20 Pass! Distance: 0.013254218562186209
No.21 Pass! Distance: 0.008738021642181912
No.22 Pass! Distance: 0.00833590771069309
No.23 Pass! Distance: 

No.197 Pass! Distance: 0.002777898266507405
No.198 Pass! Distance: 0.004110937770771654
No.199 Pass! Distance: 0.0299079675560025
No.200 Pass! Distance: 0.0628155067645825
No.201 Pass! Distance: 0.006942921761838489
No.202 Pass! Distance: 0.011029878626517033
No.203 Pass! Distance: 0.009655171392509168
No.204 Pass! Distance: 0.015104515570010581
No.205 Pass! Distance: 0.029214705340009136
No.206 Pass! Distance: 0.028441980859936777
No.207 Pass! Distance: 0.004988955424352869
No.208 Pass! Distance: 0.005524192882182585
No.209 Pass! Distance: 0.004159420710601108
No.210 Pass! Distance: 0.004188669159934689
No.211 Pass! Distance: 0.006609715954819178
No.212 Pass! Distance: 0.00823534295468458
No.213 Pass! Distance: 0.00690447364110934
No.214 Pass! Distance: 0.0062237108735305816
No.215 Pass! Distance: 0.0054675341304240505
No.216 Pass! Distance: 0.009965842646899044
No.217 Pass! Distance: 0.013050504264937911
No.218 Pass! Distance: 0.019002876879451847
No.219 Pass! Distance: 0.01552360491

No.395 Pass! Distance: 0.008950902937059598
No.396 Pass! Distance: 0.00454183130997312
No.397 Pass! Distance: 0.006932471881274565
No.398 Pass! Distance: 0.002626100569206362
No.399 Pass! Distance: 0.005223929333477611
No.400 Pass! Distance: 0.004231458521097293
No.401 Pass! Distance: 0.0035693438214019343
No.402 Pass! Distance: 0.004466565968825393
No.403 Pass! Distance: 0.004239231447226228
No.404 Pass! Distance: 0.006642744611122406
No.405 Pass! Distance: 0.0025296576396803207
No.406 Pass! Distance: 0.002514586551854438
No.407 Pass! Distance: 0.0026485948351800265
No.408 Pass! Distance: 0.009717222889738204
No.409 Pass! Distance: 0.0036627001588792753
No.410 Pass! Distance: 0.007725651441631774
No.411 Pass! Distance: 0.009563451182068737
No.412 Pass! Distance: 0.009220263447703797
No.413 Pass! Distance: 0.009258315846901365
No.414 Pass! Distance: 0.003337121465321597
No.415 Pass! Distance: 0.011215397676602117
No.416 Pass! Distance: 0.003932146647017824
No.417 Pass! Distance: 0.0030

No.594 Pass! Distance: 0.005631867929721991
No.595 Pass! Distance: 0.00531518434220304
No.596 Pass! Distance: 0.0064252578008708065
No.597 Pass! Distance: 0.008359297758106721
No.598 Pass! Distance: 0.00525064611915877
No.599 Pass! Distance: 0.006883212728012002
No.600 Pass! Distance: 0.007017857149421353
No.601 Pass! Distance: 0.008626859625496312
No.602 Pass! Distance: 0.011334981209266633
No.603 Pass! Distance: 0.010336004908713807
No.604 Pass! Distance: 0.007097923280223012
No.605 Pass! Distance: 0.012954883509398222
No.606 Pass! Distance: 0.006093516741130794
No.607 Pass! Distance: 0.006034156953760466
No.608 Pass! Distance: 0.006363874570296016
No.609 Pass! Distance: 0.0055262356882061926
No.610 Pass! Distance: 0.005942250439073419
No.611 Pass! Distance: 0.00690938525112278
No.612 Pass! Distance: 0.005232385414517081
No.613 Pass! Distance: 0.004991082496173054
No.614 Pass! Distance: 0.008764311312338787
No.615 Pass! Distance: 0.006608452179586809
No.616 Pass! Distance: 0.00384716

No.793 Pass! Distance: 0.014197565443608074
No.794 Pass! Distance: 0.008889947463164124
No.795 Pass! Distance: 0.005330394909980004
No.796 Pass! Distance: 0.003990396093151854
No.797 Pass! Distance: 0.007877012099179561
No.798 Pass! Distance: 0.005405453956111852
No.799 Pass! Distance: 0.02921624491729503
No.800 Pass! Distance: 0.03160967192969053
No.801 Pass! Distance: 0.0032567621560047966
No.802 Pass! Distance: 0.004051481976881365
No.803 Pass! Distance: 0.02568514426816751
No.804 Pass! Distance: 0.014012068641266607
No.805 Pass! Distance: 0.011801868732964347
No.806 Pass! Distance: 0.015000897929914749
No.807 Pass! Distance: 0.01301073921157902
No.808 Pass! Distance: 0.018394763737087254
No.809 Pass! Distance: 0.025136182496070587
No.810 Pass! Distance: 0.015926211065865302
No.811 Pass! Distance: 0.008660868788927398
No.812 Pass! Distance: 0.009502910201003833
No.813 Pass! Distance: 0.004813494792400744
No.814 Pass! Distance: 0.0052666945806280315
No.815 Pass! Distance: 0.010062650

...

No.12496 Pass! Distance: 0.007687207964089572
No.12497 Pass! Distance: 0.011077506035570468
No.12498 Pass! Distance: 0.01247764117197158
No.12499 Pass! Distance: 0.009750916625772066
No.12500 Pass! Distance: 0.006142073669785738
No.12501 Pass! Distance: 0.024396909152520855
No.12502 Pass! Distance: 0.0065668632773486935
No.12503 Pass! Distance: 0.00698704683450478
No.12504 Pass! Distance: 0.012859803542442304
No.12505 Pass! Distance: 0.007906778290313722
No.12506 Pass! Distance: 0.009565405535338294
No.12507 Pass! Distance: 0.006410283707236705
No.12508 Pass! Distance: 0.012883986096328456
No.12509 Pass! Distance: 0.007090377332803697
No.12510 Pass! Distance: 0.005844616639965669
No.12511 Pass! Distance: 0.00484183800627677
No.12512 Pass! Distance: 0.008529466917734029
No.12513 Pass! Distance: 0.012011534557880524
No.12514 Pass! Distance: 0.01380673682089304
No.12515 Pass! Distance: 0.009394140377962974
No.12516 Pass! Distance: 0.010303751563984928
No.12517 Pass! Distance: 0.0107099765

No.12684 Pass! Distance: 0.031201544834398534
No.12685 Pass! Distance: 0.05792625580427958
No.12686 Pass! Distance: 0.020754463824046263
No.12687 Pass! Distance: 0.0311982169799056
No.12688 Pass! Distance: 0.025543546909010083
No.12689 Pass! Distance: 0.014617255129945227
No.12690 Pass! Distance: 0.008003775757253613
No.12691 Pass! Distance: 0.0073004281514096265
No.12692 Pass! Distance: 0.013947481751961014
No.12693 Pass! Distance: 0.013002039271738456
No.12694 Pass! Distance: 0.01270205977678134
No.12695 Pass! Distance: 0.00796437924042053
No.12696 Pass! Distance: 0.0182349489957846
No.12697 Pass! Distance: 0.012353306867436981
No.12698 Pass! Distance: 0.01382776861629825
No.12699 Pass! Distance: 0.028377847500812285
No.12700 Pass! Distance: 0.01479852952871741
No.12701 Pass! Distance: 0.013429265849750505
No.12702 Pass! Distance: 0.010847641121181553
No.12703 Pass! Distance: 0.0066210940027552605
No.12704 Pass! Distance: 0.06520608390000233
No.12705 Pass! Distance: 0.038192157766034

No.12864 Pass! Distance: 0.016338608170132874
No.12865 Pass! Distance: 0.021284331068137315
No.12866 Pass! Distance: 0.0092407359149305
No.12867 Pass! Distance: 0.006831712957864644
No.12868 Pass! Distance: 0.011060546642887786
No.12869 Pass! Distance: 0.008297500377003737
No.12870 Pass! Distance: 0.017445354812733356
No.12871 Pass! Distance: 0.02332665469285056
No.12872 Pass! Distance: 0.018671557199754343
No.12873 Pass! Distance: 0.008483369735242899
No.12874 Pass! Distance: 0.007593459544072951
No.12875 Pass! Distance: 0.009103204323561993
No.12876 Pass! Distance: 0.010770843329759527
No.12877 Pass! Distance: 0.007284227589079513
No.12878 Pass! Distance: 0.004031225603133138
No.12879 Pass! Distance: 0.011826871651313972
No.12880 Pass! Distance: 0.020221503021848896
No.12881 Pass! Distance: 0.02403977275674008
No.12882 Pass! Distance: 0.014695949806829115
No.12883 Pass! Distance: 0.030585626174188325
No.12884 Pass! Distance: 0.029018535658042195
No.12885 Pass! Distance: 0.02113072819

No.13054 Pass! Distance: 0.02401887584370242
No.13055 Pass! Distance: 0.01879413897349534
No.13056 Pass! Distance: 0.013513904776491177
No.13057 Pass! Distance: 0.010872268438595787
No.13058 Pass! Distance: 0.006537600511857762
No.13059 Pass! Distance: 0.010890625366822015
No.13060 Pass! Distance: 0.00399544097265833
No.13061 Pass! Distance: 0.007620046634229241
No.13062 Pass! Distance: 0.011842727315066158
No.13063 Pass! Distance: 0.014114608420753445
No.13064 Pass! Distance: 0.012523318924047974
No.13065 Pass! Distance: 0.013137423552877024
No.13066 Pass! Distance: 0.010018940080273138
No.13067 Pass! Distance: 0.008544927693752516
No.13068 Pass! Distance: 0.017523716787493093
No.13069 Pass! Distance: 0.008239886302552527
No.13070 Pass! Distance: 0.007651449569141985
No.13071 Pass! Distance: 0.01411526810397295
No.13072 Pass! Distance: 0.012873135766922204
No.13073 Pass! Distance: 0.008822666124776462
No.13074 Pass! Distance: 0.09793305376268616
No.13075 Pass! Distance: 0.022123559761

No.13233 Pass! Distance: 0.008975640151589918
No.13234 Pass! Distance: 0.012218306530429423
No.13235 Pass! Distance: 0.0030991738481004508
No.13236 Pass! Distance: 0.011863819500189226
No.13237 Pass! Distance: 0.01570612409554106
No.13238 Pass! Distance: 0.014118417029862497
No.13239 Pass! Distance: 0.008853078660889288
No.13240 Pass! Distance: 0.006258952351207645
No.13241 Pass! Distance: 0.041517251311023226
No.13242 Pass! Distance: 0.046536455701107676
No.13243 Pass! Distance: 0.012556078105805231
No.13244 Pass! Distance: 0.012630059118977369
No.13245 Pass! Distance: 0.008938946436055922
No.13246 Pass! Distance: 0.04794609064387664
No.13247 Pass! Distance: 0.035813332736864545
No.13248 Pass! Distance: 0.016476463711370994
No.13249 Pass! Distance: 0.02900837899147371
No.13250 Pass! Distance: 0.01742868484063122
No.13251 Pass! Distance: 0.0028657905845390717
No.13252 Pass! Distance: 0.013878814649859935
No.13253 Pass! Distance: 0.012795185286610252
No.13254 Pass! Distance: 0.013307915

### Evaluation

In [20]:
print("The time of execution of above program is :", end-start)
fw.write(f"The time of execution of above program is :{end-start}")

The time of execution of above program is : 235.78010416030884


61

In [21]:
obj_ids = [0,1,2,4,5,6,8,9,10,11,12,13,14]
for i in range(len(obj_ids)):
    print(f"obj_id: {obj_ids[i]}; Success Rate: {100*less_than_two[i] / total[i]}")
    fw.write(f"obj_id: {obj_ids[i]}; Success Rate: {100*less_than_two[i] / total[i]}")

obj_id: 0; Success Rate: 93.89895138226883
obj_id: 1; Success Rate: 73.42386032977691
obj_id: 2; Success Rate: 84.01960784313725
obj_id: 4; Success Rate: 81.69291338582677
obj_id: 5; Success Rate: 93.21357285429141
obj_id: 6; Success Rate: 56.590683845391474
obj_id: 8; Success Rate: 93.52112676056338
obj_id: 9; Success Rate: 99.81185324553151
obj_id: 10; Success Rate: 99.9034749034749
obj_id: 11; Success Rate: 83.53948620361561
obj_id: 12; Success Rate: 75.48518896833504
obj_id: 13; Success Rate: 66.79462571976967
obj_id: 14; Success Rate: 72.23823246878001


In [22]:
print("Overall success rate ", 100 * sum(less_than_two) / sum(total))
fw.write(f"Overall success rate {100 * sum(less_than_two) / sum(total)}")

Overall success rate  82.74395702775291


38

In [23]:
fw.close()

In [30]:
print("Num preds diff: ", less_than_two_old - less_than_two)

Num preds diff:  [ 39. 201. 135. 127.  38. 300.  32.   2.   0. 120. 212. 306. 239.   0.
   0.]


In [31]:
print("Percentage Diff: ", 100*(less_than_two_old - less_than_two) / total)


Percentage Diff:  [ 3.7178265  19.49563531 13.23529412 12.5         3.79241517 29.73240833
  3.00469484  0.18814675  0.         11.41769743 21.65474974 29.36660269
 22.95869356         nan         nan]


/home/amoghtiwari/miniconda3/envs/densefusion/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.
